In [1]:
from pathlib import Path
import torch

cmip_path = Path("/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg")   # replace with path
era_path = Path("/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg")

In [2]:
import xarray as xr

import sys
sys.path.insert(0, '/home/seongbin/climate-learn/src')

In [3]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import DataModule

In [4]:
era5_data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = era_path,
    in_vars = ["temperature", "geopotential", "2m_temperature"],
    out_vars = ["temperature_850", "geopotential_500", "2m_temperature"],
    train_start_year = Year(1979),
    val_start_year = Year(2012),
    test_start_year = Year(2013),
    end_year = Year(2014),
    pred_range = Days(3),
    subsample = Hours(6),
    batch_size = 32,
    num_workers = 64
) 

Creating train dataset


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 46.45it/s]


out 289272
in 289272
Finished inp and out _data
Almost done!
Creating val dataset


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47.56it/s]

out 8784


in 8784
Finished inp and out _data
Almost done!
Creating test dataset


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 45.60it/s]

out 17520


in 17520
Finished inp and out _data
Almost done!


In [5]:
era5_nb = len(era5_data_module.train_dataloader())
era5_wepochs = 1000 // era5_nb + 1
era5_wepochs

1

In [7]:
cmip6_data_module = DataModule(
    dataset = "CMIP6",
    task = "forecasting",
    root_dir = cmip_path,
    in_vars = ["temperature", "geopotential", "air_temperature"],
    out_vars = ["temperature_850", "geopotential_500", "air_temperature"],
    train_start_year = Year(1979),
    val_start_year = Year(2012),
    test_start_year = Year(2013),
    end_year = Year(2014),
    pred_range = Days(3),
    batch_size = 32,
    num_workers = 64
) 

Creating train dataset


  0%|          | 0/33 [00:00<?, ?it/s]

/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/1975*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/1975*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/1975*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/1980*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/1980*.nc
/data

  0%|          | 0/1 [00:00<?, ?it/s]

/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/2010*.nc
in 1459
out 1459
Creating test dataset


  0%|          | 0/2 [00:00<?, ?it/s]

/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/temperature/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/geopotential/2010*.nc
/data0/datasets/weatherbench/data/esgf/cmip6/5.625deg/air_temperature/2010*.nc
in 2918
out 2918


In [8]:
cmip6_nb = len(cmip6_data_module.train_dataloader())
cmip6_wepochs = 1000 // cmip6_nb + 1
cmip6_wepochs, len(era5_data_module.train_dataloader()), len(cmip6_data_module.train_dataloader())

(1, 1505, 1504)

In [9]:
from climate_learn.models import load_model

In [10]:
cmip_model_kwargs = {
    "in_channels": len(cmip6_data_module.hparams.in_vars),
    "out_channels": len(cmip6_data_module.hparams.out_vars),
    "n_blocks": 19
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": cmip6_wepochs,
    "max_epochs": 100,
}

cmip_model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = cmip_model_kwargs, optim_kwargs = optim_kwargs)

In [11]:
from climate_learn.models import set_climatology
from climate_learn.training import Trainer, WandbLogger

In [12]:
set_climatology(cmip_model_module, cmip6_data_module)

In [13]:
cmip_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 100,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [14]:
cmip_trainer.fit(cmip_model_module, cmip6_data_module)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                        ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                         │ ResNet            │  5.7 M │
│ 1   │ net.activation              │ LeakyReLU         │      0 │
│ 2   │ net.image_proj              │ PeriodicConv2D    │ 18.9 K │
│ 3   │ net.image_proj.padding      │ PeriodicPadding2D │      0 │
│ 4   │ net.image_proj.conv         │ Conv2d            │ 18.9 K │
│ 5   │ net.blocks                  │ ModuleList        │  5.6 M │
│ 6   │ net.blocks.0                │ ResidualBlock     │  295 K │
│ 7   │ net.blocks.0.activation     │ LeakyReLU         │      0 │
│ 8   │ net.blocks.0.conv1          │ PeriodicConv2D    │  147 K │
│ 9   │ net.blocks.0.conv1.padding  │ PeriodicPadding2D │      0 │
│ 10  │ net.blocks.0.conv1.conv     │ Conv2d            │  147 K │
│ 11  │ net.blocks.0.conv2          │ PeriodicConv2D    │  147 K │
│ 12  │ net.blocks.0.conv2.padding  │ PeriodicPadding2D │      0 │
│ 13  │ net.blocks.0.conv2.conv     │ Conv2d            │  147 K │
│ 14  │ net.blocks.0.shortcut       │ Identity          │      0 │
│ 15  │ net.blocks.0.norm1          │ BatchNorm2d       │    256 │
│ 16  │ net.blocks.0.norm2          │ BatchNorm2d       │    256 │
│ 17  │ net.blocks.0.drop           │ Dropout           │      0 │
│ 18  │ net.blocks.1                │ ResidualBlock     │  295 K │
│ 19  │ net.blocks.1.activation     │ LeakyReLU         │      0 │
│ 20  │ net.blocks.1.conv1          │ PeriodicConv2D    │  147 K │
│ 21  │ net.blocks.1.conv1.padding  │ PeriodicPadding2D │      0 │
│ 22  │ net.blocks.1.conv1.conv     │ Conv2d            │  147 K │
│ 23  │ net.blocks.1.conv2          │ PeriodicConv2D    │  147 K │
│ 24  │ net.blocks.1.conv2.padding  │ PeriodicPadding2D │      0 │
│ 25  │ net.blocks.1.conv2.conv     │ Conv2d            │  147 K │
│ 26  │ net.blocks.1.shortcut       │ Identity          │      0 │
│ 27  │ net.blocks.1.norm1          │ BatchNorm2d       │    256 │
│ 28  │ net.blocks.1.norm2          │ BatchNorm2d       │    256 │
│ 29  │ net.blocks.1.drop           │ Dropout           │      0 │
│ 30  │ net.blocks.2                │ ResidualBlock     │  295 K │
│ 31  │ net.blocks.2.activation     │ LeakyReLU         │      0 │
│ 32  │ net.blocks.2.conv1          │ PeriodicConv2D    │  147 K │
│ 33  │ net.blocks.2.conv1.padding  │ PeriodicPadding2D │      0 │
│ 34  │ net.blocks.2.conv1.conv     │ Conv2d            │  147 K │
│ 35  │ net.blocks.2.conv2          │ PeriodicConv2D    │  147 K │
│ 36  │ net.blocks.2.conv2.padding  │ PeriodicPadding2D │      0 │
│ 37  │ net.blocks.2.conv2.conv     │ Conv2d            │  147 K │
│ 38  │ net.blocks.2.shortcut       │ Identity          │      0 │
│ 39  │ net.blocks.2.norm1          │ BatchNorm2d       │    256 │
│ 40  │ net.blocks.2.norm2          │ BatchNorm2d       │    256 │
│ 41  │ net.blocks.2.drop           │ Dropout           │      0 │
│ 42  │ net.blocks.3                │ ResidualBlock     │  295 K │
│ 43  │ net.blocks.3.activation     │ LeakyReLU         │      0 │
│ 44  │ net.blocks.3.conv1          │ PeriodicConv2D    │  147 K │
│ 45  │ net.blocks.3.conv1.padding  │ PeriodicPadding2D │      0 │
│ 46  │ net.blocks.3.conv1.conv     │ Conv2d            │  147 K │
│ 47  │ net.blocks.3.conv2          │ PeriodicConv2D    │  147 K │
│ 48  │ net.blocks.3.conv2.padding  │ PeriodicPadding2D │      0 │
│ 49  │ net.blocks.3.conv2.conv     │ Conv2d            │  147 K │
│ 50  │ net.blocks.3.shortcut       │ Identity          │      0 │
│ 51  │ net.blocks.3.norm1          │ BatchNorm2d       │    256 │
│ 52  │ net.blocks.3.norm2          │ BatchNorm2d       │    256 │
│ 53  │ net.blocks.3.drop           │ Dropout           │      0 │
│ 54  │ net.blocks.4                │ ResidualBlock     │  295 K │
│ 55  │ net.blocks.4.activation     │ LeakyReLU         │      0 │
│ 56  │ net.blocks.4.conv1          │ PeriodicC

Trainable params: 5.7 M                                                                      
Non-trainable params: 0                                                                      
Total params: 5.7 M                                                                          
Total estimated model params size (MB): 11

Output()

In [15]:
from climate_learn.models import fit_lin_reg_baseline

In [16]:
fit_lin_reg_baseline(cmip_model_module, cmip6_data_module, reg_hparam=0.0)

## Train on ERA5

In [17]:
era5_model_kwargs = {
    "in_channels": len(era5_data_module.hparams.in_vars),
    "out_channels": len(era5_data_module.hparams.out_vars),
    "n_blocks": 19
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": era5_wepochs,
    "max_epochs": 100,
}

era5_model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = era5_model_kwargs, optim_kwargs = optim_kwargs)

In [18]:
set_climatology(era5_model_module, era5_data_module)

In [19]:
era5_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 100,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [23]:
era5_trainer.fit(era5_model_module, era5_data_module)

/home/seongbin/miniconda3/envs/climate_pretrain/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /data0/ckpts/seongbin/data-cross-train/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     ┃ Name                        ┃ Type              ┃ Params ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0   │ net                         │ ResNet            │  5.7 M │
│ 1   │ net.activation              │ LeakyReLU         │      0 │
│ 2   │ net.image_proj              │ PeriodicConv2D    │ 18.9 K │
│ 3   │ net.image_proj.padding      │ PeriodicPadding2D │      0 │
│ 4   │ net.image_proj.conv         │ Conv2d            │ 18.9 K │
│ 5   │ net.blocks                  │ ModuleList        │  5.6 M │
│ 6   │ net.blocks.0                │ ResidualBlock     │  295 K │
│ 7   │ net.blocks.0.activation     │ LeakyReLU         │      0 │
│ 8   │ net.blocks.0.conv1          │ PeriodicConv2D    │  147 K │
│ 9   │ net.blocks.0.conv1.padding  │ PeriodicPadding2D │      0 │
│ 10  │ net.blocks.0.conv1.conv     │ Conv2d            │  147 K │
│ 11  │ net.blocks.0.conv2          │ PeriodicConv2D    │  147 K │
│ 12  │ net.blocks.0.conv2.padding  │ PeriodicPadding2D │      0 │
│ 13  │ net.blocks.0.conv2.conv     │ Conv2d            │  147 K │
│ 14  │ net.blocks.0.shortcut       │ Identity          │      0 │
│ 15  │ net.blocks.0.norm1          │ BatchNorm2d       │    256 │
│ 16  │ net.blocks.0.norm2          │ BatchNorm2d       │    256 │
│ 17  │ net.blocks.0.drop           │ Dropout           │      0 │
│ 18  │ net.blocks.1                │ ResidualBlock     │  295 K │
│ 19  │ net.blocks.1.activation     │ LeakyReLU         │      0 │
│ 20  │ net.blocks.1.conv1          │ PeriodicConv2D    │  147 K │
│ 21  │ net.blocks.1.conv1.padding  │ PeriodicPadding2D │      0 │
│ 22  │ net.blocks.1.conv1.conv     │ Conv2d            │  147 K │
│ 23  │ net.blocks.1.conv2          │ PeriodicConv2D    │  147 K │
│ 24  │ net.blocks.1.conv2.padding  │ PeriodicPadding2D │      0 │
│ 25  │ net.blocks.1.conv2.conv     │ Conv2d            │  147 K │
│ 26  │ net.blocks.1.shortcut       │ Identity          │      0 │
│ 27  │ net.blocks.1.norm1          │ BatchNorm2d       │    256 │
│ 28  │ net.blocks.1.norm2          │ BatchNorm2d       │    256 │
│ 29  │ net.blocks.1.drop           │ Dropout           │      0 │
│ 30  │ net.blocks.2                │ ResidualBlock     │  295 K │
│ 31  │ net.blocks.2.activation     │ LeakyReLU         │      0 │
│ 32  │ net.blocks.2.conv1          │ PeriodicConv2D    │  147 K │
│ 33  │ net.blocks.2.conv1.padding  │ PeriodicPadding2D │      0 │
│ 34  │ net.blocks.2.conv1.conv     │ Conv2d            │  147 K │
│ 35  │ net.blocks.2.conv2          │ PeriodicConv2D    │  147 K │
│ 36  │ net.blocks.2.conv2.padding  │ PeriodicPadding2D │      0 │
│ 37  │ net.blocks.2.conv2.conv     │ Conv2d            │  147 K │
│ 38  │ net.blocks.2.shortcut       │ Identity          │      0 │
│ 39  │ net.blocks.2.norm1          │ BatchNorm2d       │    256 │
│ 40  │ net.blocks.2.norm2          │ BatchNorm2d       │    256 │
│ 41  │ net.blocks.2.drop           │ Dropout           │      0 │
│ 42  │ net.blocks.3                │ ResidualBlock     │  295 K │
│ 43  │ net.blocks.3.activation     │ LeakyReLU         │      0 │
│ 44  │ net.blocks.3.conv1          │ PeriodicConv2D    │  147 K │
│ 45  │ net.blocks.3.conv1.padding  │ PeriodicPadding2D │      0 │
│ 46  │ net.blocks.3.conv1.conv     │ Conv2d            │  147 K │
│ 47  │ net.blocks.3.conv2          │ PeriodicConv2D    │  147 K │
│ 48  │ net.blocks.3.conv2.padding  │ PeriodicPadding2D │      0 │
│ 49  │ net.blocks.3.conv2.conv     │ Conv2d            │  147 K │
│ 50  │ net.blocks.3.shortcut       │ Identity          │      0 │
│ 51  │ net.blocks.3.norm1          │ BatchNorm2d       │    256 │
│ 52  │ net.blocks.3.norm2          │ BatchNorm2d       │    256 │
│ 53  │ net.blocks.3.drop           │ Dropout           │      0 │
│ 54  │ net.blocks.4                │ ResidualBlock     │  295 K │
│ 55  │ net.blocks.4.activation     │ LeakyReLU         │      0 │
│ 56  │ net.blocks.4.conv1          │ PeriodicC

Trainable params: 5.7 M                                                                      
Non-trainable params: 0                                                                      
Total params: 5.7 M                                                                          
Total estimated model params size (MB): 11

Output()

In [24]:
fit_lin_reg_baseline(era5_model_module, era5_data_module, reg_hparam=0.0)

# Data

Days: 5, Var: temp 850, model: resnet

train_start_year = 1979 / 1979, val_start_year = 2012, test_start_year = 2013, end_year = 2014

In [25]:
# cmip -> era
cmip_trainer.test(cmip_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/acc_2m_temperature_day_3        │             0.8107210703657786              │
│       test/acc_geopotential_500_day_3       │             0.48953478376442333             │
│       test/acc_temperature_850_day_3        │             0.6107390599467657              │
│      test/w_rmse_2m_temperature_day_3       │             3.3500194130329586              │
│     test/w_rmse_geopotential_500_day_3      │             1010.1774194620833              │
│      test/w_rmse_temperature_850_day_3      │              4.286434214356342              │
│ test_persistence_baseline/w_rmse_2m_temper… │              4.539468185748448              │
│ test_persistence_baseline/w_rmse_geopotent… │             1131.7358679580643              │
│ test_persistence_baseline/w_rmse_temperatu… │              5.110162641210476              │
│ test_ridge_regression_baseline/w_rmse_2m_t… │              78.45677302864252              │
│ test_ridge_regression_baseline/w_rmse_geop… │              40249.90188265724              │
│ test_ridge_regression_baseline/w_rmse_temp… │              65.19058513485234              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [26]:
# era -> era
era5_trainer.test(era5_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/acc_2m_temperature_day_3        │             0.8381540307283244              │
│       test/acc_geopotential_500_day_3       │             0.5541038881354717              │
│       test/acc_temperature_850_day_3        │             0.6749115243032309              │
│      test/w_rmse_2m_temperature_day_3       │             2.8984720029997706              │
│     test/w_rmse_geopotential_500_day_3      │              834.6526452506582              │
│      test/w_rmse_temperature_850_day_3      │             3.5750410151385323              │
│ test_persistence_baseline/w_rmse_2m_temper… │              4.460446957206656              │
│ test_persistence_baseline/w_rmse_geopotent… │             1124.7930564391704              │
│ test_persistence_baseline/w_rmse_temperatu… │              4.957310029289739              │
│ test_ridge_regression_baseline/w_rmse_2m_t… │             15.429813134100284              │
│ test_ridge_regression_baseline/w_rmse_geop… │             2070.8682377607124              │
│ test_ridge_regression_baseline/w_rmse_temp… │             13.505711190012667              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [22]:
# cmip -> cmip
cmip_trainer.test(cmip_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_air_temperature_day_3        │              0.943422429245578              │
│       test/acc_geopotential_500_day_3       │             0.9640222720595856              │
│       test/acc_temperature_850_day_3        │             0.9384346455672873              │
│      test/w_rmse_air_temperature_day_3      │              1.723953769153641              │
│     test/w_rmse_geopotential_500_day_3      │             312.54694089968086              │
│      test/w_rmse_temperature_850_day_3      │             1.8014642702876726              │
│ test_persistence_baseline/w_rmse_air_tempe… │              4.250243936938531              │
│ test_persistence_baseline/w_rmse_geopotent… │             1220.3341714837843              │
│ test_persistence_baseline/w_rmse_temperatu… │              5.010411231265007              │
│ test_ridge_regression_baseline/w_rmse_air_… │              78.53497865903158              │
│ test_ridge_regression_baseline/w_rmse_geop… │             40276.621985431484              │
│ test_ridge_regression_baseline/w_rmse_temp… │              65.27743891915748              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [27]:
# era -> cmip
era5_trainer.test(era5_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_air_temperature_day_3        │             0.8316408819007174              │
│       test/acc_geopotential_500_day_3       │             0.5522263061352618              │
│       test/acc_temperature_850_day_3        │              0.693716597671046              │
│      test/w_rmse_air_temperature_day_3      │              3.03910333647438               │
│     test/w_rmse_geopotential_500_day_3      │              919.0756713192048              │
│      test/w_rmse_temperature_850_day_3      │              3.637307642947853              │
│ test_persistence_baseline/w_rmse_air_tempe… │              4.176257752189843              │
│ test_persistence_baseline/w_rmse_geopotent… │              1212.847953716597              │
│ test_persistence_baseline/w_rmse_temperatu… │              4.860542384032741              │
│ test_ridge_regression_baseline/w_rmse_air_… │             15.377150814108784              │
│ test_ridge_regression_baseline/w_rmse_geop… │             2116.8605641188215              │
│ test_ridge_regression_baseline/w_rmse_temp… │              13.48245629237431              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘